# Converged and Non-converged Simulation

## Import dependencies

In [52]:
import networkx as nx
import torch
import torch.nn as nn
import random
import time
import bisect
import operator
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## Constants and data containers

In [53]:
# Generation
GENERATOR = None
GENERATOR_PATH = "generator5.pt"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
RANDOM_SEED = 77

# Network & Flows
NETWORK = None
BLOCKED = None # count of flows blocked
SENT = None # count of flows that reached their destination
ADDRESSES = [i for i in range(1,22)]
WAVELENGTHS = 40
EXPON_SCALE = 0.1
EXPON_DIST = np.random.exponential(scale=EXPON_SCALE, size=20000000)

# Simulation
SIM_CLOCK = None 
MIN_FLOWS_SENT = 1e2
EVENTS_LIST = None
ARCHITECTURE = None 

IAT_TIMES = []
DURATIONS = []

## Classes

In [54]:
class Generator(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(2, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 2),
        )

    def forward(self, x):
        output = self.model(x)
        return output

In [55]:
class ConvergedNetwork:
    def __init__(self, graph, num_of_wavelengths):
        self.topology = graph
        self.__allocate_capacity__(num_of_wavelengths)
    
    
    def __allocate_capacity__(self, num_of_wavelengths):
        self.links = dict()
        for edge in self.topology.edges():
            node1, node2 = edge[0], edge[1] 
            self.links[(node1, node2)] = num_of_wavelengths 
       
    
    def __check_capacity__(self, node1, node2): 
        try:
            return self.links[(node1, node2)] > 0
        except KeyError:
            return self.links[(node2, node1)] > 0
    
    
    def __use_capacity__(self, node1, node2):
        try: 
            self.links[(node1, node2)] -= 1
        except KeyError:
            self.links[(node2, node1)] -= 1

            
    def __release_capacity__(self, node1, node2):
        try:   
            self.links[(node1, node2)] += 1
        except KeyError:
            try:
                self.links[(node2, node1)] += 1
            except KeyError: # One of the nodes must be None
                pass # there is no link to release capacity for
                
        
    def push_flow(self, flow, type):
        node1, node2 = flow.current_node, flow.route[0]
        if type == "HOP":
            self.__release_capacity__(flow.prev_node, flow.current_node)
            
        sufficient_capacity = self.__check_capacity__(node1, node2)
        if sufficient_capacity:
            self.__use_capacity__(node1, node2)
            next_event_type = flow.hop()
            return next_event_type, flow 
        else:
            next_event_type = "BLOCKED"
            return next_event_type, flow
            
            
    def end_flow(self, flow):
        node1, node2 = flow.prev_node, flow.current_node
        self.__release_capacity__(node1, node2)
        # print(f"Sim Clock: {SIM_CLOCK} seconds | Flow {flow.ID} has reached its destination, node {flow.dst}. Now leaving the network...")


    def find_route(self, src, dst):
        return nx.shortest_path(self.topology, source = src, target = dst)

In [56]:
class NonConvergedNetwork:
    def __init__(self, graph, num_of_wavelengths):
        self.topology = graph
        self.__allocate_capacity__(num_of_wavelengths)
    
    
    def __allocate_capacity__(self, num_of_wavelengths):
        self.links = dict()
        for edge in self.topology.edges():
            node1, node2 = edge[0], edge[1]
            # Each item in the list will represent a unique wavelength in the link
            self.links[(node1, node2)] = [format(i, "02") for i in range(1, num_of_wavelengths+1)]
            
    
    def __check_capacity__(self, node1, node2):
        # Return all the available wavelengths
        try:
            return self.links[(node1, node2)]
        except KeyError:
            return self.links[(node2, node1)]
    
    
    def __use_capacity__(self, node1, node2, wavelength):
        try: 
            self.links[(node1, node2)].remove(wavelength)
        except KeyError:
            self.links[(node2, node1)].remove(wavelength) 

            
    def __release_capacity__(self, node1, node2, wavelength):
        try:   
            self.links[(node1, node2)].append(wavelength)
        except KeyError:
            self.links[(node2, node1)].append(wavelength)
            
                
    def find_route(self, src, dst):
        return nx.shortest_path(self.topology, source = src, target = dst)
    
    
    def __wavelength_assignment__(self, flow):
        wavelength_counter = dict()
        viable_lightpaths = []

        for link in flow.lightpath:
            available_wavelengths = self.__check_capacity__(link[0], link[1])
            for wavelength in available_wavelengths:
                wavelength_counter[wavelength] = wavelength_counter.get(wavelength,0) + 1

        for key in wavelength_counter.keys():
            if wavelength_counter[key] == len(flow.lightpath): # Pick the first wavelength that is available on all the links in the path
                flow.wavelength = key
                return flow
            
        flow.wavelength = "00"
        return flow
        
    def push_flow(self, flow):
        updated_flow = self.__wavelength_assignment__(flow)
        if updated_flow.wavelength == "00":
            return "BLOCKED", updated_flow
        
        else:
            # Uses lightpath from flow to update the capacity on each link in the lightpath
            for link in updated_flow.lightpath:
                self.__use_capacity__(link[0], link[1], updated_flow.wavelength)
            return "DEPART", updated_flow

        
    def end_flow(self, flow):
        for link in flow.lightpath:
            self.__release_capacity__(link[0], link[1], flow.wavelength)
        # print(f"Sim Clock: {SIM_CLOCK} seconds | Flow {flow.ID} has reached its destination, node {flow.dst}. Now leaving the network...")

In [57]:
class ConvergedFlow:
    def __init__(self):
        # self.parent_net = network
        self.__create_flow__()
        
    def __generate_data__(self):
        torch.manual_seed(RANDOM_SEED)
        random_noise = torch.randn((1, 2), device = DEVICE)
        generated_samples = GENERATOR(random_noise)
        generated_samples = generated_samples.cpu().detach().numpy()
        synthetic_dur, synthetic_size = generated_samples[0,0], generated_samples[0,1]
        return synthetic_dur, synthetic_size
        
    def __create_convflow__(self):
        self.dur, self.size = self.__generate_data__()
        random_addresses = random.sample(ADDRESSES, 2)
        self.src, self.dst = random_addresses[0], random_addresses[1]
        self.current_node = self.src
        self.prev_node = None
        
        # Calculate shortest path
        self.route = NETWORK.find_route(self.src, self.dst)
        
        # Removes the first node in the route which is the starting node for the flow path 
        self.route = self.route[1:] 
        
        # How long it will take to make each hop, used to schedule the next event time
        self.hop_time = self.dur / len(self.route) 
        
        
    def hop(self): # Pushes the flow into the network from src to dst/from one hop to the next
        self.prev_node = self.current_node 
        # print(f"Sim Clock: {SIM_CLOCK} seconds | Flow {self.ID} is moving from node {self.current_node} to node {self.route[0]}.")
        self.current_node = self.route.pop(0)
        
        if len(self.route) == 0:
            # print(f"Flow {self.ID} has reached its destination, node {self.dst}. Now leaving the network...")
            next_event_type = "DEPART"
        else:
            next_event_type = "HOP"
        
        return next_event_type

In [58]:
class NonConvergedFlow:
    def __init__(self):
        self.__create_flow__()
        
    def __generate_data__(self):
        torch.manual_seed(RANDOM_SEED)
        random_noise = torch.randn((1, 2), device = DEVICE)
        generated_samples = GENERATOR(random_noise)
        generated_samples = generated_samples.cpu().detach().numpy()
        synthetic_dur, synthetic_size = generated_samples[0,0], generated_samples[0,1]
        return synthetic_dur, synthetic_size
        
    def __create_flow__(self):
        self.wavelength = ""
        self.dur, self.size = self.__generate_data__()
        random_addresses = random.sample(ADDRESSES, 2)
        self.src, self.dst = random_addresses[0], random_addresses[1]
        
        # Calculate shortest path
        self.route = NETWORK.find_route(self.src, self.dst)
        
        # Define a lightpath for the flow from its node pairs
        self.lightpath = []
        for i in range(len(self.route)):
            try:
                self.lightpath.append((self.route[i], self.route[i+1]))
            except IndexError: # Gotten to the end of the path list and has accounted for all the links
                pass  

In [59]:
class Event:
    def __init__(self, event_time, event_type, flow):
        self.event_type = event_type # ARRIVAL, DEPARTURE, HOP OR BLOCKED
        self.event_time = event_time # When the event will happen in simulated time
        self.associated_flow = flow # The same flow is attached to an Event object, only type and time changes throughout the simulation

## Functions

In [60]:
def schedule_event(event_time, event_type, associated_flow):
    global EVENTS_LIST
    
    # Create new event
    new_event = Event(event_time, event_type, associated_flow)
    
    # Create key with which the events list will be sorted; will be sorted by event time
    time_key = operator.attrgetter("event_time")
    
    # Inserts a new event into the Events list and maintains its sorted order
    bisect.insort(EVENTS_LIST, new_event, key=time_key) 

In [61]:
def initialise_simulation():
    global SIM_CLOCK, GENERATOR, NETWORK, BLOCKED, SENT, EVENTS_LIST, ARCHITECTURE
    
    EVENTS_LIST = []
    # Set simulation clock to 0
    SIM_CLOCK = 0
    # global sim_clock, generator, network, BLOCKED, SENT

    # Initialise the generator
    GENERATOR = Generator()
    GENERATOR.load_state_dict(torch.load(GENERATOR_PATH, map_location=DEVICE))
    GENERATOR = GENERATOR.to(DEVICE)
    
    # Initialise the network
    G = nx.read_adjlist("UKnet", nodetype=int)
    
    NETWORK = ConvergedNetwork(G, WAVELENGTHS) if ARCHITECTURE=="CONVERGED" else NonConvergedNetwork(G, WAVELENGTHS)
    
    # Initialise statistical counters
    BLOCKED = 0
    SENT = 0

    # Add one event to the list to start the simulation
    new_flow = ConvergedFlow() if ARCHITECTURE=="CONVERGED" else NonConvergedFlow()
    DURATIONS.append(new_flow.dur)
    schedule_event(event_time = 0, event_type = "ARRIVAL", associated_flow = new_flow)

In [62]:
# Check the next event in the list and advance simulation clock
def timing_routine():
    global SIM_CLOCK
    # Popping the next event from the top of the list
    next_event = EVENTS_LIST.pop(0)
    
    # Accessing the time of the next scheduled event
    advanced_time = next_event.event_time
    
    # Advancing the simulation clock to the time of the next scheduled event
    SIM_CLOCK = advanced_time
    
    return next_event

In [63]:
def event_routine(event):
    global SIM_CLOCK, NETWORK, SENT, BLOCKED, ARCHITECTURE

    current_flow = event.associated_flow
    
    if event.event_type == "ARRIVAL" or event.event_type == "HOP":
        
        if ARCHITECTURE=="CONVERGED":
            # Allow the network to push the flow to the next node
            next_event_type, updated_flow = NETWORK.push_flow(current_flow, event.event_type)
        
            if next_event_type != "BLOCKED": # if blocked, there's no new event to schedule and capacity in the previous link has been released.
                schedule_event(SIM_CLOCK+updated_flow.hop_time, next_event_type, associated_flow = updated_flow)
            else:
                BLOCKED += 1
                
        else:
            next_event_type, updated_flow = NETWORK.push_flow(current_flow)
            if next_event_type == "DEPART":
                schedule_event(SIM_CLOCK + updated_flow.dur, "DEPART", updated_flow) 
            else:
                BLOCKED += 1

    else: # Departure event
        NETWORK.end_flow(current_flow)
        SENT += 1
    
    # Only generating new arrival events when the current event is an arrival so that the iat is used correctly.
    if event.event_type == "ARRIVAL":
        
    # Generate a future arrival event and add to the events list
        new_flow = ConvergedFlow() if ARCHITECTURE=="CONVERGED" else NonConvergedFlow()
        np.random.seed(RANDOM_SEED)
        iat = np.random.choice(EXPON_DIST)
        IAT_TIMES.append(iat) # To calculate the average IAT to calculate the load (Erlangs)
        schedule_event(event_time = SIM_CLOCK + iat, event_type = "ARRIVAL", associated_flow = new_flow)

In [64]:
def convert_time(seconds):
    seconds = seconds % (24 * 3600)
    hour = seconds // 3600
    seconds %= 3600
    minutes = seconds // 60
    seconds %= 60
     
    return "%d hours %02d mins %02d secs" % (hour, minutes, seconds)

In [65]:
def report(run_time):
    global IAT_TIMES, DURATIONS
    sim_time = convert_time(run_time)
    blocked_flows = round(BLOCKED*100/(BLOCKED+SENT), 4)
    sent_flows = 100 - blocked_flows
    
    IAT_TIMES = np.array(IAT_TIMES)
    mean_iat  = np.mean(IAT_TIMES)
    
    DURATIONS = np.array(DURATIONS)
    mean_dur  = np.mean(DURATIONS)
    
    print(f"The simulation took {sim_time} to complete")
    print(f"Total flows travelled through the network: {BLOCKED + SENT}")
    print(f"{sent_flows} % of all flows arrived successfully to its destination, {SENT} flows")
    print(f"{blocked_flows} % of all flows were blocked due to lack of capacity in a link, {BLOCKED} flows")
    print()
    print(f"The average interarrival time of a flow was {round(mean_iat,3)} seconds")
    print(f"The average duration of a flow was {round(mean_dur,3)} seconds")
    print(f"The traffic load of this simulation was {round(mean_dur/mean_iat,3)} erlangs")

In [66]:
def choose_architecture():
    print("What kind of architecture would you like to simulate today?")
    print("1. Converged\n2. Non-converged")
    print("Enter '1' for converged or '2' for non-converged")
    while True:
        try:
            architecture = int(input())
            if 0 < architecture < 3:
                break
            else:
                print("Choice must be 1 or 2. Try again.")
        except:
            print("Choice must be an integer. Try again.")
            
    return "CONVERGED" if (architecture==1) else "NONCONVERGED"

In [67]:
# Main function for testing purposes
def main():
    ARCHITECTURE = choose_architecture()
    print(F"Starting {ARCHITECTURE} simulation...")
    start_time = time.time()
    initialise_simulation()
    
    progress = 0
    progress_bar = tqdm(total=MIN_FLOWS_SENT, desc="Simulating", unit = " flows sent", leave=False)
    
    while (SENT + BLOCKED) < MIN_FLOWS_SENT:
        next_event = timing_routine()
        event_routine(next_event)
        progress = SENT + BLOCKED
        progress_bar.update(progress - progress_bar.n)
        
    end_time = time.time()
    run_time = end_time - start_time
    report(run_time)

In [68]:
if __name__ == "__main__":
    main()

What kind of architecture would you like to simulate today?
1. Converged
2. Non-converged
Enter '1' for converged or '2' for non-converged
2
Starting NONCONVERGED simulation...


The simulation took 0 hours 00 mins 01 secs to complete
Total flows travelled through the network: 100
0.0 % of all flows arrived successfully to its destination, 0 flows
100.0 % of all flows were blocked due to lack of capacity in a link, 100 flows

The average interarrival time of a flow was 0.066 seconds
The average duration of a flow was 117.10700225830078 seconds
The traffic load of this simulation was 1784.324 erlangs
